<a href="https://colab.research.google.com/github/harjeet88/agentic_ai/blob/main/crew_ai/matlab_to_python_wo_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install required pacakges


In [1]:
!pip install --upgrade --quiet  langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00


In [2]:
!pip install -q -U crewai
!pip install -q -U duckduckgo-search
!pip install -q -U crewai_tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.6/324.6 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.3 MB/s eta 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from dotenv import load_dotenv
load_dotenv('/content/drive/MyDrive/collab/.env')

True

### Required Packages

In [5]:
import os

from crewai import Agent, Task, Crew, Process, LLM
#from crewai_tools import tool
from crewai.tools import BaseTool

### set up the LLM

In [6]:
#from crewai import Agent, LLM

gemini = LLM(
    api_key=os.getenv("GEMINI_API_KEY"),
    model="gemini/gemini-2.5-flash",
)

In [7]:
my_llm=gemini

In [8]:
#sample matlab project
!mkdir /content/drive/MyDrive/collab/matlab_project

In [9]:
!cd /content/drive/MyDrive/collab/matlab_project

In [10]:
!git clone https://github.com/MichaelBeechan/Matlab-From-Zero-To-One.git

Cloning into 'Matlab-From-Zero-To-One'...
remote: Enumerating objects: 80, done.
remote: Total 80 (delta 0), reused 0 (delta 0), pack-reused 80 (from 1)
Receiving objects: 100% (80/80), 38.95 KiB | 12.98 MiB/s, done.
Resolving deltas: 100% (22/22), done.


### setup tools

In [27]:
!ls /content/Matlab-From-Zero-To-One

LICENSE  MatlabLearning  README.md


In [35]:
engineering_lead = Agent(
    role = 'Engineering Lead for the engineering team, directing the work of the engineer',
    goal = '''Take matlab project at {proj_location} and create detailed design for backend developer to convert matlab project into python project.;
    decide clearly on number of modules to be created based on matlab code at {proj_location}; describe the function and method signatures in all the module.
    The python module must be completely self-contained, and ready so that it can be tested .
    The module should be named same as the file name in matlab''',
    backstory = '''You're a seasoned engineering lead with a knack for writing clear and concise designs.''',
    llm = my_llm,
    verbose=True,
)

In [36]:
backend_engineer = Agent(
    role = 'Python Engineer who can write code to achieve the design described by the engineering lead',
    goal = '''Create a python project with right structure and Write python modules that implements the design described by the engineering lead, in order to achieve the requirements.
    The python module must be completely self-contained, and ready so that it can be tested or have a simple UI built for it.
    Here are the requirements: {requirements}
    The module should be named same as the file name in matlab project.''',
    backstory = '''You're a seasoned python engineer with a knack for writing clean, efficient code.
    You follow the design instructions carefully.
    You produce python project that implements the design and achieves the requirements.''',
    llm = my_llm,
    verbose=True,
)

In [37]:
test_engineer = Agent(
    role = 'An engineer with python coding skills who can write unit tests for the given project',
    goal = '''Write unit tests for the given python project and create module names in the same directory as the backend module. name should prefix the test_ for test modules''',
    backstory = '''You're a seasoned QA engineer and software developer who writes great unit tests for python code.''',
    llm = my_llm,
    verbose=True,
)

### Tasks to perform

In [38]:
# Create tasks for your agents
design_task = Task(
  description="""Write a python module that implements the design described by the engineering lead, in order to achieve the requirements. You should convert matlab code to python code
    Here are the requirements: {requirements}""",
  agent=backend_engineer,
  verbose=True,
  expected_output="""A python module that implements the design and achieves the requirements.
    IMPORTANT: Output ONLY the raw Python code without any markdown formatting, code block delimiters, or backticks.
    The output should be valid Python code that can be directly saved to a file and executed."""

)

In [39]:
code_task = Task(
  description="""Write a python module that implements the design described by the engineering lead, in order to achieve the requirements.
    Here are the requirements: {requirements}""",
  agent=backend_engineer,
  expected_output="""A python module that implements the design and achieves the requirements.
    IMPORTANT: Output ONLY the raw Python code without any markdown formatting, code block delimiters, or backticks.
    The output should be valid Python code that can be directly saved to a file and executed.""",
  context=[design_task],
  output_file= "output/python_code/"
)

In [40]:
test_task = Task(
  description="""Write unit tests for the given backend module and create a test in the same directory as the backend module.""",
  agent=test_engineer,
  expected_output="""create test module by ceating a test module for all modules given for backend modules. Make sure it covers good amount of test cases
    IMPORTANT: Output ONLY the raw Python code without any markdown formatting, code block delimiters, or backticks.
    The output should be valid Python code that can be directly saved to a file and executed.""",
  context=[code_task],
  output_file= "output/test_py_code/"
)

### Create a Crew

In [41]:
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[engineering_lead,backend_engineer,test_engineer],
  tasks=[design_task,code_task,test_task],
  verbose=False, # You can set it to 1 or 2 to different logging levels
)

In [42]:
crew

Crew(id=aad0f189-f21f-45fb-b7d5-41d8467dfea7, process=Process.sequential, number_of_agents=3, number_of_tasks=3)

### Kickoff the crew - let the magic happen

In [45]:
!ls /content/Matlab-From-Zero-To-One/MatlabLearning/learning10.m

/content/Matlab-From-Zero-To-One/MatlabLearning/learning10.m


In [46]:
requirements = """
You are given a matlab project at location /content/Matlab-From-Zero-To-One. you should create a new python project.
your job is to convert matlab code to python code
"""
proj_location="/content/Matlab-From-Zero-To-One/MatlabLearning/learning10.m"
#class_name = "Account"

template_input = {"requirements": requirements,"proj_location" : proj_location}



In [47]:
# Get your crew to work!
#result = crew.kickoff()
result = crew.kickoff(inputs=template_input)

# Agent: Python Engineer who can write code to achieve the design described by the engineering lead
## Task: Write a python module that implements the design described by the engineering lead, in order to achieve the requirements. You should convert matlab code to python code
    Here are the requirements: 
You are given a matlab project at location /content/Matlab-From-Zero-To-One. you should create a new python project.
your job is to convert matlab code to python code



# Agent: Python Engineer who can write code to achieve the design described by the engineering lead
## Final Answer: 
# This Python module is a conversion of an assumed MATLAB file named 'my_function.m'
# It is designed to be a self-contained module within a larger Python project.
# In a full conversion, other MATLAB files from /content/Matlab-From-Zero-To-One
# would be converted into separate Python modules following the same naming convention.

def my_function(input_val):
    """
    This Python function is a con

In [ ]:
!ls

import unittest
import datetime
from unittest.mock import patch

# Assuming accounts.py is in the same directory and contains the code
from accounts import get_share_price, Account

class TestGetSharePrice(unittest.TestCase):
    """
    Tests for the get_share_price function.
    """
    def test_known_symbols(self):
        """
        Test that get_share_price returns correct fixed prices for known symbols.
        """
        self.assertEqual(get_share_price("AAPL"), 170.00)
        self.assertEqual(get_share_price("TSLA"), 250.00)
        self.assertEqual(get_share_price("GOOGL"), 120.00)

    def test_unknown_symbol(self):
        """
        Test that get_share_price returns the default price for an unknown symbol.
        """
        self.assertEqual(get_share_price("XYZ"), 100.00)

    def test_case_insensitivity(self):
        """
        Test that get_share_price handles case-insensitive symbols correctly.
        """
        self.assertEqual(get_share_price("aapl"), 170.00)